In [1]:
import numpy as np

In [14]:
templates = {
    "症状到病":[
        {"我{sym1}而且{sym2}，是得了什么病？":"O {sym1} O O {sym2} O O O O O O O O"},
        {"{sym1}和{sym2}说明得了什么病":"{sym1} O {sym2} O O O O O O O"},
        {"{sym1}和{sym2}是什么病的症状？":"{sym1} O {sym2} O O O O O O O O"},
        {"生了什么病会{sym1}、{sym2}":"O O O O O O {sym1} O {sym2}"}
    ],
    "症状到药":[
        {"我{sym1}而且{sym2}，应该吃什么药？":"O {sym1} O O {sym2} O O O O O O O O"},
        {"{sym1}和{sym2}得吃什么治":"{sym1} O {sym2} O O O O O"},
        {"{sym1}和{sym2}吃什么能好？":"{sym1} O {sym2} O O O O O"},
        {"吃什么能治{sym1}、{sym2}":"O O O O O {sym1} O {sym2}"}
    ],
    "疾病到药":[
        {"我得了{dis1}和{dis2}，应该吃什么药？":"O O O {dis1} O {dis2} O O O O O O"},
        {"{dis1}和{dis2}这两个病得吃什么治":"{dis1} O {dis2} O O O O O O O O O"},
        {"得了{dis1}和{dis2}吃什么能好？":"O O {dis1} O {dis2} O O O O O"},
        {"吃什么能治{dis1}、{dis2}两个病":"O O O O O {dis1} O {dis2} O O O"}
    ],
    "药到病":[
        {"{drug1}和{drug2}能治疗什么病？":"{drug1} O {drug2} O O O O O O"},
        {"{drug1}和{drug2}两个药能治疗什么":"{drug1} O {drug2} O O O O O O O O"},
        {"我吃{drug1}和{drug2}能治病吗":"O O {drug1} O {drug2} O O O O"},
        {"吃{drug1}和{drug2}有效果的吧":"O {drug1} O {drug2} O O O O O"}
    ],
    "饮片比较":[
        {"{yp1}和{yp2}哪个比较甜":"{yp1} O {yp2} O O O O O"},
        {"{yp1}和{yp2}两个药材味道如何":"{yp1} O {yp2} O O O O O O O O"},
        {"{yp1}和{yp2}哪个好吃":"{yp1} O {yp2} O O O O"},
        {"{yp1}和{yp2}哪个比较苦":"{yp1} O {yp2} O O O O O"}
    ],
    "方剂组成":[
        {"{fj1}和{fj2}共同包含什么":"{fj1} O {fj2} O O O O O O"},
        {"{fj1}和{fj2}两个方剂有哪些共同点":"{fj1} O {fj2} O O O O O O O O O O"},
        {"{fj1}和{fj2}相同饮片有什么":"{fj1} O {fj2} O O O O O O O"},
        {"{fj1}和{fj2}由什么共同组成":"{fj1} O {fj2} O O O O O O O"}
    ],  
}

In [3]:
#从neo4j拿到 症状、疾病、药材、饮片、方剂
cur = ['症状','疾病','药材','饮片','方剂']
dic = {}
for i in cur:
    dic[i]=[]

from py2neo import Graph,Node,NodeMatcher,RelationshipMatcher

graph = Graph(
            host="10.15.82.65",
            port=7687,
            user="neo4j",
            )
node_matcher = NodeMatcher(graph)

for single in cur:
    nodes = node_matcher.match(single)
    for node in nodes:
        dic[single].append(node['name'])

In [54]:
SAMPLE_SIZE = 100

def get_data(templates):
    res = []
    for label in templates:
        template_choice = np.random.choice(templates[label], SAMPLE_SIZE)
        texts = [list(_.keys())[0] for _ in template_choice]
        tags = [list(_.values())[0] for _ in template_choice]
        if "{sym1}" in texts[0]:
            fit_texts = np.random.choice(dic['症状'], SAMPLE_SIZE)
            for i in range(SAMPLE_SIZE):
                texts[i] = texts[i].replace('{sym1}',fit_texts[i])
                item_len = len(fit_texts[i])
                if item_len == 1:
                    fit_tag = "B"
                else:
                    fit_tag = "B " + "I "*(item_len-1)
                tags[i] = tags[i].replace('{sym1}',fit_tag.rstrip(" "))
        if "{sym2}" in texts[0]:
            fit_texts = np.random.choice(dic['症状'], SAMPLE_SIZE)
            for i in range(SAMPLE_SIZE):
                texts[i] = texts[i].replace('{sym2}',fit_texts[i])
                item_len = len(fit_texts[i])
                if item_len == 1:
                    fit_tag = "B"
                else:
                    fit_tag = "B " + "I "*(item_len-1)
                tags[i] = tags[i].replace('{sym2}',fit_tag.rstrip(" "))
                
        if "{dis1}" in texts[0]:
            fit_texts = np.random.choice(dic['疾病'], SAMPLE_SIZE)
            for i in range(SAMPLE_SIZE):
                texts[i] = texts[i].replace('{dis1}',fit_texts[i])
                item_len = len(fit_texts[i])
                if item_len == 1:
                    fit_tag = "B"
                else:
                    fit_tag = "B " + "I "*(item_len-1)
                tags[i] = tags[i].replace('{dis1}',fit_tag.rstrip(" "))
        if "{dis2}" in texts[0]:
            fit_texts = np.random.choice(dic['疾病'], SAMPLE_SIZE)
            for i in range(SAMPLE_SIZE):
                texts[i] = texts[i].replace('{dis2}',fit_texts[i])
                item_len = len(fit_texts[i])
                if item_len == 1:
                    fit_tag = "B"
                else:
                    fit_tag = "B " + "I "*(item_len-1)
                tags[i] = tags[i].replace('{dis2}',fit_tag.rstrip(" "))
                     
        if "{drug1}" in texts[0]:
            fit_texts = np.random.choice(dic['药材'], SAMPLE_SIZE)
            for i in range(SAMPLE_SIZE):
                texts[i] = texts[i].replace('{drug1}',fit_texts[i])
                item_len = len(fit_texts[i])
                if item_len == 1:
                    fit_tag = "B"
                else:
                    fit_tag = "B " + "I "*(item_len-1)
                tags[i] = tags[i].replace('{drug1}',fit_tag.rstrip(" "))
        if "{drug2}" in texts[0]:
            fit_texts = np.random.choice(dic['药材'], SAMPLE_SIZE)
            for i in range(SAMPLE_SIZE):
                texts[i] = texts[i].replace('{drug2}',fit_texts[i])
                item_len = len(fit_texts[i])
                if item_len == 1:
                    fit_tag = "B"
                else:
                    fit_tag = "B " + "I "*(item_len-1)
                tags[i] = tags[i].replace('{drug2}',fit_tag.rstrip(" "))
                     
        if "{yp1}" in texts[0]:
            fit_texts = np.random.choice(dic['饮片'], SAMPLE_SIZE)
            for i in range(SAMPLE_SIZE):
                texts[i] = texts[i].replace('{yp1}',fit_texts[i])
                item_len = len(fit_texts[i])
                if item_len == 1:
                    fit_tag = "B"
                else:
                    fit_tag = "B " + "I "*(item_len-1)
                tags[i] = tags[i].replace('{yp1}',fit_tag.rstrip(" "))
        if "{yp2}" in texts[0]:
            fit_texts = np.random.choice(dic['饮片'], SAMPLE_SIZE)
            for i in range(SAMPLE_SIZE):
                texts[i] = texts[i].replace('{yp2}',fit_texts[i])
                item_len = len(fit_texts[i])
                if item_len == 1:
                    fit_tag = "B"
                else:
                    fit_tag = "B " + "I "*(item_len-1)
                tags[i] = tags[i].replace('{yp2}',fit_tag.rstrip(" "))   
                
        if "{fj1}" in texts[0]:
            fit_texts = np.random.choice(dic['方剂'], SAMPLE_SIZE)
            for i in range(SAMPLE_SIZE):
                texts[i] = texts[i].replace('{fj1}',fit_texts[i])
                item_len = len(fit_texts[i])
                if item_len == 1:
                    fit_tag = "B"
                else:
                    fit_tag = "B " + "I "*(item_len-1)
                tags[i] = tags[i].replace('{fj1}',fit_tag.rstrip(" "))   
        if "{fj2}" in texts[0]:
            fit_texts = np.random.choice(dic['方剂'], SAMPLE_SIZE)
            for i in range(SAMPLE_SIZE):
                texts[i] = texts[i].replace('{fj2}',fit_texts[i])
                item_len = len(fit_texts[i])
                if item_len == 1:
                    fit_tag = "B"
                else:
                    fit_tag = "B " + "I "*(item_len-1)
                tags[i] = tags[i].replace('{fj2}',fit_tag.rstrip(" "))                    
                
        for text, tag in zip(texts, tags):
            res.append([text, label, tag])
    np.random.shuffle(res)
    res = np.stack(res, axis=-1)
    train_texts, train_lables, train_tags = res
    return train_texts, train_lables, train_tags

In [55]:
train_data, train_labels, train_tags = get_data(templates)

In [56]:
import os,json
write_path = "."
clf_path = os.path.join(write_path, "clf_data.json")
ner_path = os.path.join(write_path, "ner_data.json")
if not os.path.exists(write_path):
    os.mkdir(write_path)
with open(clf_path,"w",encoding="utf-8") as f:
    json.dump({"train_data":list(train_data),"train_labels":list(train_labels)},f,ensure_ascii=False,indent=4)
with open(ner_path,"w",encoding="utf-8") as f:
    json.dump({"train_data":list(train_data),"train_tags":list(train_tags)},f,ensure_ascii=False,indent=4)

In [57]:
tem = []
for data,tag in zip(train_data, train_tags):
    tem.append([data,tag])
with open("ner_train_data.txt","w",encoding="utf-8") as f:
    json.dump(tem,f,ensure_ascii=False,indent=4)